In [6]:
import ee
import geemap

In [7]:
# Initialize the Earth Engine API
ee.Initialize()

In [ ]:
def generate_evi(bbox, start_date, end_date, cloud_cover=80):
    """
    Generate an Enhanced Vegetation Index (EVI) for a given bbox and time period.

    Parameters:
        bbox (ee.Geometry): The bbox of interest.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        cloud_cover (int): Maximum cloud cover percentage (default: 80).

    Returns:
        ee.ImageCollection: Image collection with EVI and NDVI bands.
    """
    # Load Landsat 8 SR dataset
    landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterBounds(bbox) \
        .filterDate(start_date, end_date) \
        .filterMetadata('CLOUD_COVER', 'less_than', cloud_cover)
    
    # Function to scale Landsat bands
    def scaling_ls(img):
        optical = img.select('SR_B.').multiply(0.0000275).add(-0.2)
        thermal = img.select('ST_B.*').multiply(0.00341802).add(149.0)
        return img.addBands(optical, None, True).addBands(thermal, None, True)
    
    # Function to mask clouds
    def mask_clouds(img):
        cloud_shadow_bit_mask = (1 << 4)
        clouds_bit_mask = (1 << 3)
        clouds_cirrus_bit_mask = (1 << 2)
        clouds_dilated_bit_mask = (1 << 1)
        qa = img.select('QA_PIXEL')
        mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0) \
                .And(qa.bitwiseAnd(clouds_bit_mask).eq(0)) \
                .And(qa.bitwiseAnd(clouds_cirrus_bit_mask).eq(0)) \
                .And(qa.bitwiseAnd(clouds_dilated_bit_mask).eq(0))
        return img.updateMask(mask)
    
    # Function to calculate NDVI and EVI
    def calc_vis_ls(img):
        ndvi = img.normalizedDifference(['SR_B5', 'SR_B4']).rename('ndvi')
        evi = img.expression(
            '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
                'NIR': img.select('SR_B5'),
                'RED': img.select('SR_B4'),
                'BLUE': img.select('SR_B2')
            }).rename('evi')
        return img.addBands([ndvi, evi])
    
    # Apply the scaling, cloud masking, and VI calculation
    processed = landsat.map(scaling_ls).map(mask_clouds).map(calc_vis_ls)
    return processed

In [9]:
# Example usage
california = ee.Geometry.Polygon([
    [[-124.482003, 32.529521],
     [-114.131211, 32.529521],
     [-114.131211, 42.009518],
     [-124.482003, 42.009518],
     [-124.482003, 32.529521]]
])


In [10]:
evi_collection = generate_evi(bbox=california, start_date='2021-07-01', end_date='2021-07-31')

# Visualize EVI on a map
Map = geemap.Map()
Map.centerObject(california, 6)
evi_vis = {'min': 0, 'max': 1, 'palette': ['white', 'blue', 'green']}
ndvi_vis = {'min': 0, 'max': 1, 'palette': ['white', 'yellow', 'green']}

Map.addLayer(evi_collection.select('evi').median(), evi_vis, 'EVI')
Map.addLayer(evi_collection.select('ndvi').median(), ndvi_vis, 'NDVI')

Map

Map(center=[37.28134182218032, -119.306607], controls=(WidgetControl(options=['position', 'transparent_bg'], w…